In [153]:
%pip install opencv-python-headless==4.9.0.80
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import re
import json
import pandas as pd
from datetime import datetime
print('install successfull')

Note: you may need to restart the kernel to use updated packages.
install successfull


In [291]:
BASE_PATH = Path('/home/jovyan/work/videos/')
VIDEO_PATHS = list(BASE_PATH.rglob('*.MP4')) + list(BASE_PATH.rglob('*.mp4'))
CSV_PATHS = list(BASE_PATH.rglob('*.csv'))

ELAN = '_ELAN'
SYNC = '_sync'

In [293]:
len(VIDEO_PATHS)

37

In [363]:
def find_files(video_paths=VIDEO_PATHS):
    files_dict = {}

    for v in tqdm(video_paths):
        name = v.stem
        folder = v.parent.parent
        
        csvs = [str(x) for x in folder.iterdir() if x.is_file() and x.suffix == '.csv']
        match_csv = [file for file in csvs if re.search(name, file)]
        if match_csv:
            match_csv = match_csv[0]
        else:
            match_csv = None

        videoanalyse_folder = Path(folder / 'Videoanalyse')
        if videoanalyse_folder.exists() and videoanalyse_folder.is_dir():
            txt_name = name[:13] + ELAN + name[13:] + SYNC
            txts = [str(x) for x in videoanalyse_folder.iterdir() if x.is_file() and x.suffix == '.txt']
            match_txt = [file for file in txts if re.search(txt_name, file)]
            if match_txt:
                match_txt = match_txt[0]
            else:
                match_txt = None
        
            files_dict[str(name)] = {
                'start_end_file': match_csv,
                'analysis_file': match_txt,
                'path': str(v),
                'start_time': 0,
                'end_time': 0    
            }
    output_dir = Path('/home/jovyan/work/output')
    if not output_dir.exists():
        output_dir.mkdir(parents=True, exist_ok=True)

    with open(str(output_dir / 'files.json'), 'w') as json_file:
        json.dump(files_dict, json_file, indent=4)
        
    return files_dict



def get_times(files_dict):
    files_dict = files_dict
    
    for file, file_info in tqdm(files_dict.items()):
        se_file = file_info.get('start_end_file')
        if se_file is None:
            print(f"Warning: start_end_file not found for {file}. Skipping.")
            continue

        df = pd.read_csv(se_file)
        
        start = str(df.iloc[0]['hour']) + ':' + str(df.iloc[0]['minute']) + ':' + str(df.iloc[0]['milisecond'])[0:2] + '.' + str(df.iloc[0]['milisecond'])[2:-1]
        end = str(df.iloc[1]['hour']) + ':' + str(df.iloc[1]['minute']) + ':' + str(df.iloc[1]['milisecond'])[0:2] + '.' + str(df.iloc[1]['milisecond'])[2:-1]

        files_dict[str(file)]['start_time'] = start
        files_dict[str(file)]['end_time'] = end
        
    output_dir = Path('/home/jovyan/work/output')
    if not output_dir.exists():
        output_dir.mkdir(parents=True, exist_ok=True)

    with open(str(output_dir / 'files.json'), 'w') as json_file:
        json.dump(files_dict, json_file, indent=4)

    return files_dict



def get_analysis(files_dict):

    for file, file_info in tqdm(files_dict.items()):
        sync_file = file_info['analysis_file']

        if sync_file is not None:
            try:
                columns = ['category', 'file', 'timestamp_start_long', 'timestamp_start_short', 
                   'timestamp_end_long', 'timestamp_end_short', 'length_long', 'length_short', 
                   'label', 'label_str']
                df = pd.read_csv(sync_file, sep="	", header=None)
                df.columns = columns
                filtered_df = df[df['category'].str.contains('SE')]
            
                output_dir = Path('/home/jovyan/work/output/' + file)
                if not output_dir.exists():
                    output_dir.mkdir(parents=True, exist_ok=True)
                res_path = str(output_dir / 'analysis.csv')
                filtered_df.to_csv(res_path, index=False)
        
                file_info['analysis_file'] = res_path
            except pd.errors.ParserError as e:
                print(f"Error parsing CSV file: {e} with file {sync_file}")
                continue

        else: 
            pass

    output_dir = Path('/home/jovyan/work/output')
    if not output_dir.exists():
        output_dir.mkdir(parents=True, exist_ok=True)

    with open(str(output_dir / 'files.json'), 'w') as json_file:
        json.dump(files_dict, json_file, indent=4)
    return files_dict

In [409]:
d = find_files()
e = get_times(d)
f_dict = get_analysis(e)

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 251.14it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 105.88it/s]


In [410]:
times_dict = {
    'START_S001_T1_La1': "16:30:56.00",
    'START_S001_T1_La2': "16:55:21.00",
    'START_S001_T1_LaCon_Ki': "17:12:57.00",
    'START_S001_T1_LaCon_Mu': "17:22:38.00",
    
    'START_S004_T1_La1': "00:00:00.00",
    'START_S004_T1_La2':  "00:00:00.00",
    'START_S004_T1_LaCon_Ki':  "00:00:00.00",
    'START_S004_T1_LaCon_Mu':  "00:00:00.00",
    
    'START_S005_T1_La1':  "17:12:00.00",
    'START_S005_T1_La2':  "00:00:00.00",
    'START_S005_T1_LaCon_Ki':  "00:00:00.00",
    'START_S005_T1_LaCon_Mu_cut':  "00:00:00.00",
    
    'START_S006_T1_La1': "15:06:05.00",
    'START_S006_T1_La2': "15:28:55.00",
    'START_S006_T1_LaCon_Ki': "15:48:04.00",
    'START_S006_T1_LaCon_Mu': "15:55:54.00",
    
    'START_S002_T1': "16:59:00.00",
    'START_S002_T1_La1': "00:00:00.00",
    'START_S002_T1_La2': "00:00:00.00"
}


In [411]:
f_dict

{'START_S001_T1_La1': {'start_end_file': '/home/jovyan/work/videos/S001/T1/START_S001_T1_La1.csv',
  'analysis_file': '/home/jovyan/work/output/START_S001_T1_La1/analysis.csv',
  'path': '/home/jovyan/work/videos/S001/T1/Videos/START_S001_T1_La1.MP4',
  'start_time': '16:31:23.92',
  'end_time': '16:36:23.94'},
 'START_S001_T1_La2': {'start_end_file': '/home/jovyan/work/videos/S001/T1/START_S001_T1_La2.csv',
  'analysis_file': '/home/jovyan/work/output/START_S001_T1_La2/analysis.csv',
  'path': '/home/jovyan/work/videos/S001/T1/Videos/START_S001_T1_La2.MP4',
  'start_time': '16:56:15.9',
  'end_time': '17:1:16.0'},
 'START_S001_T1_LaCon_Ki': {'start_end_file': '/home/jovyan/work/videos/S001/T1/START_S001_T1_LaCon_Ki.csv',
  'analysis_file': None,
  'path': '/home/jovyan/work/videos/S001/T1/Videos/START_S001_T1_LaCon_Ki.MP4',
  'start_time': '17:13:41.69',
  'end_time': '17:18:41.69'},
 'START_S001_T1_LaCon_Mu': {'start_end_file': '/home/jovyan/work/videos/S001/T1/START_S001_T1_LaCon_Mu

In [458]:
def save_video_frames(frames, output_file, width, height):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('/home/jovyan/work/clipped.mp4', fourcc, 25, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()

def clip_and_save_video(file_info, times_dict):
    video_time = datetime.strptime(times_dict[file], '%H:%M:%S.%f')
    start_time = datetime.strptime(file_info['start_time'], '%H:%M:%S.%f')
    diff = (start_time - video_time).seconds

    

    cap = cv2.VideoCapture(str(file_info['path']))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    idx = int(fps * diff)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    output_file = "clipped.mp4"
    save_video_frames(frames, output_file, width, height)

In [459]:
for file, file_info in tqdm(f_dict.items()):
    print(file)
    clip_and_save_video(file_info, times_dict)
    break
print('finished')

  0%|                                                                                            | 0/19 [00:00<?, ?it/s]

START_S001_T1_La1


  0%|                                                                                            | 0/19 [00:11<?, ?it/s]


In [439]:
for file, file_info in tqdm(f_dict.items()):
    print(file)
    video_time = datetime.strptime(times_dict[file], '%H:%M:%S.%f')
    start_time = datetime.strptime(file_info['start_time'], '%H:%M:%S.%f')
    diff = (start_time - video_time).seconds

    cap = cv2.VideoCapture(str(file_info['path']))
    fps, num_frames = int(cap.get(cv2.CAP_PROP_FPS)), int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    idx = (fps*diff)
        
    for f in (range(num_frames)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        _, frame = cap.read()
        show_image(frame)
        break
    
    break
        
    
    
    
    """
    #print(v)
    c
    
    print('fps: ',fps)
    print('num_frames: ', num_frames)

    sec = 3.5

    for f in (range(num_frames)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, fps*sec)
        _, frame = cap.read()
        show_image(frame)
        break
    break
    #show_image()
    
    
  
    for f in tqdm(rangenum_frames)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f)
        ret, frame = cap.read()
        res_path = './' + str(path) + f'/frame_{f}.jpg'
        save_image(res_path, frame)
    """

  0%|                                                                                            | 0/19 [00:00<?, ?it/s]

START_S001_T1_La1


  0%|                                                                                            | 0/19 [00:02<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:

# Dataset-specific paths
CURRENT_PATH = BASE_PATH / DATA
LABELS = [f.name for f in CURRENT_PATH.iterdir() if f.is_dir()]
IMAGE_PATHS = list(CURRENT_PATH.rglob('*.jpg'))


### Define paths

In [3]:
in_path = Path('./data/in/videos')
out_path = Path('./data/out/videos')

### Utilities

In [5]:
def manage_and_create_paths(image_path, out_path):
    res = out_path / image_path.stem
    if not res.parent.exists():
        res.mkdir(parents=True, exist_ok=True)
    return res

def create_output_path(path, out_path):
    return str(manage_and_create_paths(path, out_path)) + '/' + path.name

### Extract frames

In [19]:
for p in (in_path.rglob('*.mp4')):
    print(p)
    path = manage_and_create_paths(p, out_path)
    cap = cv2.VideoCapture(str(p))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print('frames per second: ',fps)
    l = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('total number of frames: ', l)
    
    for f in tqdm(range(l)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f)
        ret, frame = cap.read()
        res_path = './' + str(path) + f'/frame_{f}.jpg'
        save_image(res_path, frame)

frames per second:  29.97002997002997
total number of frames:  2196


100%|██████████| 2196/2196 [00:44<00:00, 49.80it/s]


## In case multiprocessing works:

In [ ]:
import cv2
from pathlib import Path
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

def process_video(video_path, out_path):
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)
    l = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    for f in range(l):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f)
        ret, frame = cap.read()
        frames.append((f, frame))
    cap.release()

    path = manage_and_create_paths(video_path, out_path)
    for frame_info in frames:
        f, frame = frame_info
        res_path = str(path) + f'/frame_{f}.jpg'
        save_image(res_path, frame)

def process_videos(video_paths, out_path):
    with Pool(cpu_count()) as pool:
        pool.starmap(process_video, [(video_path, out_path) for video_path in video_paths])

# Assuming manage_and_create_paths and save_image are defined elsewhere

# Main code
video_paths = list(in_path.rglob('*.mp4'))
process_videos(video_paths, out_path)
